# Факультативное задание

**Задание.**

Удалите из events события для редких айтемов — таких, с которыми взаимодействовало менее N пользователей.

Возьмите небольшое N, например 2–3 пользователя.

Получите рекомендации, посчитайте метрики, оцените, как они изменились.

Подумайте, с чем могут быть связаны такие изменения.

# Initialization

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [3]:
import surprise

In [4]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

In [5]:
def mdir(obj):
    print(*[i for i in dir(obj) if not i.startswith("_")], sep="\n")

# Загрузка данных

In [6]:
events = pd.read_parquet("events.par")

In [8]:
events.shape

(11751086, 9)

In [9]:
events.head()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_old
0,1000000,11012,2015-12-05,2015-12-11,True,4,False,2015-12-01,00000377eea48021d3002730d56aca9a
1,1000000,4671,2014-06-05,2014-06-30,True,5,False,2014-06-01,00000377eea48021d3002730d56aca9a
2,1000000,5,2012-10-02,2012-10-24,True,5,False,2012-10-01,00000377eea48021d3002730d56aca9a
3,1000000,2,2009-07-12,2009-07-29,True,5,False,2009-07-01,00000377eea48021d3002730d56aca9a
4,1000000,14497,2016-05-09,2016-06-02,True,5,False,2016-05-01,00000377eea48021d3002730d56aca9a


In [10]:
events = events.groupby("item_id").filter(lambda x: x["user_id"].nunique() > 3)

In [11]:
events.shape

(11742745, 9)

In [39]:
print(f"Удалено {1 - 11742745 / 11751086:.2%}")

Удалено 0.07%


# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

В качестве точки разбиения используем `2017-08-01`, то есть отнесем в тестовую часть три последних месяца.

In [12]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = np.intersect1d(users_train, users_test)

print(len(users_train), len(users_test), len(common_users)) 

428206 123136 120765


# === Знакомство: "холодный" старт

## "Холодные" пользователи

«Холодные» пользователи — те, которые есть в `test`, но отсутствуют в `train`. Это соответствует хронологическому порядку, в котором и работает рекомендательная система.

In [13]:
cold_users = np.setdiff1d(users_test, users_train)

print(len(cold_users)) 

2371


## Топ-100 наиболее популярных книг

Теперь можно построить сами рекомендации. Для этого найдем топ-100 наиболее популярных книг (учитывая и их среднюю оценку) за последние несколько лет. Например, с 2015 года и со средней оценкой не меньше 4. Их мы будем рекомендовать «холодным» пользователям.

In [14]:
top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

In [15]:
item_popularity.head()

,item_id,users,avg_rating
0,1,8728,4.796059
1,2,9528,4.685873
2,3,15139,4.706057
3,5,11890,4.770143
4,6,10679,4.764772


In [16]:
item_popularity["popularity_weighted"] = item_popularity["users"] * item_popularity["avg_rating"]

In [17]:
# сортируем по убыванию взвешенной популярности
item_popularity = item_popularity.sort_values("popularity_weighted", ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity[item_popularity["avg_rating"] > 4].head(100)

In [18]:
top_k_pop_items

,item_id,users,avg_rating,popularity_weighted
30625,18007564,20207,4.321275,87320.0
30845,18143977,19462,4.290669,83505.0
29024,16096824,16770,4.301014,72128.0
2,3,15139,4.706057,71245.0
3670,38447,14611,4.232770,61845.0
...,...,...,...,...
18542,2767052,4361,4.413437,19247.0
31045,18293427,4674,4.092640,19129.0
376,3636,4667,4.098564,19128.0
31749,18966819,4361,4.374914,19079.0


## Рекомендации

Топ-рекомендации назовём рекомендациями по умолчанию (англ. default recommendations).

Уже можно оценить их качество — для этого их следует соотнести с событиями из теста, как будто бы вы показываете эти рекомендации «холодным» пользователям, зная их предпочтения.

In [19]:
# соотнесем события из теста с топ-рекомендациями

cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items[["item_id", "avg_rating"]], on="item_id", how="left")

In [20]:
cold_users_events_with_recs.shape

(9667, 10)

In [21]:
cold_users_events_with_recs.head()

,user_id,item_id,started_at,read_at,is_read,rating,is_reviewed,started_at_month,user_id_old,avg_rating
0,1000153,29236299,2017-08-16,2017-08-20,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
1,1000153,26114463,2017-08-20,2017-09-09,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
2,1000153,13262783,2017-08-12,2017-08-14,True,5,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
3,1000153,29923707,2017-10-08,2017-10-13,True,4,False,2017-10-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN
4,1000153,18806259,2017-08-08,2017-08-12,True,3,False,2017-08-01,001ae592ce3cdb7abb6f19b9b4d19638,NaN


In [22]:
# найдем клиентов с покрытием по рекомендациям

cold_user_recs = cold_users_events_with_recs.loc[
    cold_users_events_with_recs["avg_rating"].notna(), ["user_id", "item_id", "rating", "avg_rating"]
]

In [23]:
cold_user_recs.shape

(1912, 4)

In [24]:
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180


In [25]:
# Для какой доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items совпали по книгам

cold_users_events_with_recs["avg_rating"].notna().mean()

0.1977862832316127

## Метрики

Расчитаем метрики `rmse` и `mae` для полученных рекомендаций.

In [26]:
cold_user_recs.head()

,user_id,item_id,rating,avg_rating
13,1000504,1885,5,4.316316
17,1000712,13496,5,4.440779
23,1001508,1885,5,4.316316
25,1002222,18966819,5,4.374914
27,1002222,15839976,4,4.150180


In [27]:
rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])

print("RMSE =", round(rmse, 3))
print(" MAE =", round(mae, 3))

RMSE = 0.776
 MAE = 0.625


### Покрытие холодных пользователей рекомендациями

Полученная метрика — значение, на которое оценка рекомендации в среднем отклоняется от истинной

Это отклонение выглядит небольшим, но в данном случае стоит оценить покрытие как по пользователям, так и по объектам, ведь рекомендации, скорее всего, имеют мало пересечений с оценками пользователей. 

In [28]:
# посчитаем покрытие холодных пользователей рекомендациями

cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (x.notna()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio['hits'] == 0).mean():.2f}")
print(f"Доля пользователей с релевантными рекомендациями: {(cold_users_hit_ratio['hits'] > 0).mean():.2f}")
print(f"\nСреднее покрытие пользователей: {cold_users_hit_ratio.loc[cold_users_hit_ratio['hits'] != 0, 'hits'].mean():.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Доля пользователей с релевантными рекомендациями: 0.41

Среднее покрытие пользователей: 0.44


«Холодных» пользователей без каких-либо релевантных рекомендаций — 59%, то есть пересечение между оценёнными книгами и рекомендациями есть только у 41%, по ним же и получено значение MAE-метрики. При этом среднее покрытие — 44%. Это значит, что большая часть «холодных» пользователей не получила никаких релевантных рекомендаций, а оставшаяся часть имеет пересечения только по 44% книг. 

# === Знакомство: первые персональные рекомендации

Степень разреженности матрицы оценок

In [29]:
# Степень разреженности матрицы можно оценить как отношение незаполненных ячеек к общему их количеству:

sparsity = 1 - events[["user_id", "item_id"]].drop_duplicates().shape[0] / (events["user_id"].nunique() * events["item_id"].nunique())
sparsity


0.9992666223698916

## Реализация SVD-алгоритма

Воспользуемся готовой реализацией SVD-алгоритма из библиотеки `surprise`. В качестве разбиения данных на `train` и `test` возьмём разбиение из предыдущего урока: `events_train`, `events_test`.

In [31]:
# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = surprise.Reader(rating_scale=(1, 5))
surprise_train_set = surprise.Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)

In [32]:
surprise_train_set = surprise_train_set.build_full_trainset()

In [33]:
# инициализируем модель
svd_model = surprise.SVD(n_factors=100, random_state=0)

In [34]:
# обучаем модель
svd_model.fit(surprise_train_set)

Используя обученную модель, получим рекомендации для тестовой выборки:

In [35]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set) 

## Оценка рекомендаций

Полученные рекомендации можно оценить, используя встроенный модуль `accuracy` из библиотеки `surprise`:

In [36]:
rmse = surprise.accuracy.rmse(svd_predictions)
mae = surprise.accuracy.mae(svd_predictions)

print()
print(rmse, mae) 

RMSE: 0.8268
MAE:  0.6464

0.8268059754278789 0.6463998925172703


**Значения на полном наборе данных:**

RMSE: 0.8263<br/>
MAE:  0.6460

0.826346375350908 0.6460143973270805

# Выводы:

- Метрики по предсказаниям из топ-книг не изменились
  - Удаление редких книг не влияет на топ-книги
- Метрики SVD-алгоритма стали чуть хуже, но незначительно
  - Удалено немного данных (менее 0,1%), плэтому влияние несущественно
  - Т.е. событий по редким книгам значительно меньше, чем по популярным
